In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import backend as K
from keras.models import Model
from keras.optimizers import SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint
from IDNNs.idnns.information.information_process import get_information
from IDNNs.idnns.plots.plot_figures import plot_all_epochs, extract_array, load_figures
from joblib import dump, load

Using TensorFlow backend.


In [2]:

AES_Sbox = np.array(
    [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76, 0xCA, 0x82, 0xC9,
     0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0, 0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F,
     0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15, 0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07,
     0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75, 0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3,
     0x29, 0xE3, 0x2F, 0x84, 0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58,
     0xCF, 0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8, 0x51, 0xA3,
     0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2, 0xCD, 0x0C, 0x13, 0xEC, 0x5F,
     0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73, 0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88,
     0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB, 0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC,
     0x62, 0x91, 0x95, 0xE4, 0x79, 0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A,
     0xAE, 0x08, 0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A, 0x70,
     0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E, 0xE1, 0xF8, 0x98, 0x11,
     0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF, 0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42,
     0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16])
AES_inv_Sbox = np.array(
    [0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB, 0x7C, 0xE3, 0x39,
     0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB, 0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2,
     0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E, 0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76,
     0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25, 0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC,
     0x5D, 0x65, 0xB6, 0x92, 0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D,
     0x84, 0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06, 0xD0, 0x2C,
     0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B, 0x3A, 0x91, 0x11, 0x41, 0x4F,
     0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73, 0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85,
     0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E, 0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62,
     0x0E, 0xAA, 0x18, 0xBE, 0x1B, 0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD,
     0x5A, 0xF4, 0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F, 0x60,
     0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF, 0xA0, 0xE0, 0x3B, 0x4D,
     0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61, 0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6,
     0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D])

hw = np.array([bin(x).count("1") for x in range(256)])


In [3]:
def load_traces(database_file, start_at=1, number_samples=0):
    # Power Consumption Start_at -> Start_at + Number_Samples
    traces = np.loadtxt(database_file, delimiter=',', dtype=np.float64, skiprows=1,
                        usecols=range(start_at, start_at + number_samples))
    # Plaintext Start_at - 1
    inputoutput = np.loadtxt(database_file, delimiter=',', dtype=np.str, skiprows=1,
                             usecols=start_at - 1)
    # Chiphertext Start_at + Number_Samples
    # Key Start_at + Number_Samples + 1
    key = np.loadtxt(database_file, delimiter=',', dtype=np.str, skiprows=1,
                             usecols=start_at + number_samples + 1)
    # print("traces shape: {}\ninputoutput shape: {}\n".format(traces.shape, inputoutput.shape))
    return traces, inputoutput, key


def shorten_traces(dataset, start_at=0, number_samples=0):
    if len(dataset) == 3:
        traces, inputoutput, key = dataset
    elif len(dataset) == 4:
        traces, inputoutput, key, labels = dataset
        labels_selected = labels[start_at:start_at + number_samples]
        
    traces_selected = traces[start_at:start_at + number_samples]
    inputoutput_selected = inputoutput[start_at:start_at + number_samples]
    key_selected = key[start_at:start_at + number_samples]
    
    if len(dataset) == 3:
        return traces_selected, inputoutput_selected, key_selected
    elif len(dataset) == 4:
        return traces_selected, inputoutput_selected, key_selected, labels_selected


def statcorrect_traces(dataset):
    if len(dataset) == 3:
        traces, inputoutput, key = dataset
    elif len(dataset) == 4:
        traces, inputoutput, key, labels = dataset

    # traces_statcorrect = (traces - np.mean(traces, axis=1).reshape(-1,1))/np.std(traces, axis=1).reshape(-1,1)
    traces_statcorrect = (traces - np.mean(traces, axis=0).reshape(1, -1)) / np.std(traces, axis=0).reshape(1, -1)

    if len(dataset) == 3:
        return traces_statcorrect, inputoutput, key
    elif len(dataset) == 4:
        return traces_statcorrect, inputoutput, key, labels

def create_labels_sboxinputkey(dataset, database_file, col):
    if len(dataset) == 3:
        traces, inputoutput, key = dataset
    else:
        traces, inputoutput, key, _ = dataset

    labels = np.loadtxt(database_file, delimiter=',', dtype=np.int, skiprows=1,
                     usecols=col)

    return traces, inputoutput, key, labels


In [5]:
# 1 Conv 1 Dense Layers with Gradient Descent Optimizer
def create_model(classes=9, number_samples=200):
    input_shape = (number_samples, 1)
    trace_input = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=10, strides=10, activation='relu', padding='valid', name='block1_conv1')(
        trace_input)
    # x = Conv1D(filters=32, kernel_size=3, strides=3, activation='relu', padding='valid', name='block1_conv2')(x)
    x = MaxPooling1D(pool_size=2, strides=2, padding='valid', name='block1_pool')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(200, activation='tanh', name='fc1')(x)
    x = Dense(128, activation='tanh', name='fc2')(x)
    x = Dense(128, activation='tanh', name='fc3')(x)
    x = Dense(64, activation='tanh', name='fc4')(x)
    x = Dense(64, activation='tanh', name='fc5')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(trace_input, x, name='cnn')
    optimizer = SGD(lr=0.001, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 6 Dense Layers with RMSprop optimizer MLP
def create_model_mlp(classes=9, number_samples=200):
    input_shape = (number_samples, 1)
    trace_input = Input(shape=input_shape)
    x = Flatten(name='flatten')(trace_input)
    x = Dense(200, activation='relu', name='fc1')(x)
    x = Dense(200, activation='relu', name='fc2')(x)
    # x = Dense(200, activation='relu', name='fc3')(x)
    x = Dense(128, activation='relu', name='fc4')(x)
    x = Dense(128, activation='relu', name='fc5')(x)
    # x = Dense(128, activation='relu', name='fc6')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    model = Model(trace_input, x, name='mlp_RMSprop')
    optimizer = RMSprop(lr=0.0001, decay=0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 1 Conv 2 Dense Layers with RMSprop optimizer CNN
def create_model3(classes=9, number_samples=200):
    input_shape = (number_samples, 1)
    trace_input = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=10, strides=10, activation='relu', padding='same', name='block1_conv1')(
        trace_input)
    x = AveragePooling1D(pool_size=2, strides=2, padding='same', name='block1_pool')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='tanh', name='fc1')(x)
    x = Dense(4096, activation='tanh', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    model = Model(trace_input, x, name='cnn_RMSprop')
    optimizer = RMSprop(lr=0.00001, decay=0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model



In [6]:
def key_rank(model, inout_test, traces_test, kByte, trueKey):
    p = model.predict(traces_test)
    rank = np.zeros(inout_test.shape[0])
    prob_vector = np.zeros(256)

    for i, v in enumerate(inout_test):
        for kh in range(0, 256):
            hemw = hw[AES_Sbox[bytes.fromhex(v)[kByte] ^ kh]]
            prob_vector[kh] += p[i][hemw]
        df = pd.DataFrame({'prob': prob_vector})
        df = df.sort_values(['prob'],ascending=False)
        df = df.reset_index()
        df.rename(columns={'index': 'keyH'},inplace=True)
        rank[i] = df[df.keyH == int(trueKey, 16)].index.tolist()[0]
    return rank


def _evaluate(model: Model, nodes_to_evaluate, x, y=None):
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, nodes_to_evaluate)
    x_, y_, sample_weight_ = model._standardize_user_data(x, y)
    return f(x_ + y_ + sample_weight_)


def get_activations(model, x, layer_name=None):
    nodes = [layer.output for layer in model.layers if layer.name == layer_name or layer_name is None]
    # we process the placeholders later (Inputs node in Keras). Because there's a bug in Tensorflow.
    input_layer_outputs, layer_outputs = [], []
    [input_layer_outputs.append(node) if 'input_' in node.name else layer_outputs.append(node) for node in nodes]
    activations = _evaluate(model, layer_outputs, x, y=None)
    activations_dict = dict(zip([output.name for output in layer_outputs], activations))
    activations_inputs_dict = dict(zip([output.name for output in input_layer_outputs], x))
    result = activations_inputs_dict.copy()
    result.update(activations_dict)
    completeActivations = []
    for i in range(len(result)):
        completeActivations.append(list(result.items())[i][1])
    return completeActivations



In [7]:
trainset = 'datasets/SmartCardAES/AES_trainset.csv'
testset = 'datasets/SmartCardAES/AES_testset.csv'

dataset = load_traces(trainset, 1, 1654)
dataset = statcorrect_traces(dataset)
dataset_test = load_traces(testset, 1, 1654)
dataset_test = statcorrect_traces(dataset_test)


In [27]:

# range 16
for i in range(1,2):
    dataset_keyh = create_labels_sboxinputkey(dataset, trainset, 1656 + i)  # Template - Use known SubKey byte
    # dataset_keyh = shorten_traces(dataset_keyh,0,20000)
    dataset_test_core = create_labels_sboxinputkey(dataset_test, testset, 1656 + i)
    #dataset_test_core = shorten_traces(dataset_test_core, 0, 20000)
    
    # dataset_keyhype = split_data_percentage(dataset_keyhype, training_fraction=0.85)
    traces_train, inputoutput_train, _, labels_train = dataset_keyh
    traces_test, inputoutput_test, key, labels_test = dataset_test_core
    print("<------------------Attacking Byte: {}------------------>".format(i))
    # print(traces_train.shape, traces_train.dtype)
    # print(traces_test.shape, traces_test.dtype)
    # print(inputoutput_train.shape, inputoutput_train.dtype)
    # print(inputoutput_test.shape, inputoutput_test.dtype)
    # print(labels_train.shape, labels_train.dtype)
    # print(labels_test.shape, labels_test.dtype)
    # print(labels_train[0])
    # min_class_tr = int(np.min(labels_train))
    # min_class_ts = int(np.min(labels_test))
    # classes = max(len(np.unique(labels_train)) + min_class_tr, len(np.unique(labels_test)) + min_class_ts)
    classes = 9
    traces_train_reshaped = traces_train.reshape((traces_train.shape[0], traces_train.shape[1], 1))
    labels_train_categorical = to_categorical(labels_train, num_classes=classes)
    traces_test_reshaped = traces_test.reshape((traces_test.shape[0], traces_test.shape[1], 1))
    labels_test_categorical = to_categorical(labels_test, num_classes=classes)
    save_model = ModelCheckpoint('model_epoch{epoch}.h5', period=100)
    
    class CalculateRecall(Callback):
        def __init__(self, data, labels, message_prefix=None):
            self.data = data
            self.labels = labels
            self.message_prefix = message_prefix + ' ' or ''

        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            predictions = self.model.predict(self.data)
            correctly_classified = (np.argmax(predictions, axis=1) == self.labels)
            _sum = 0.
            for i in np.unique(self.labels):
                n_correct = len(np.nonzero(correctly_classified[np.where(self.labels == i)[0]])[0])
                n_total = len(np.where(self.labels == i)[0])
                _sum += n_correct / n_total
            recall = _sum / len(np.unique(self.labels))
            print(self.message_prefix + 'recall:', recall)


    class CalculateActivations(Callback):
        def __init__(self, calcEpoch):
            self.calcEpoch = calcEpoch
            self.idx = 0
            self.ws = []
            
        def on_epoch_end(self, epoch, logs=None):
            if epoch == self.calcEpoch[self.idx]:
                print("Getting Activations...")
                self.ws.append(get_activations(self.model, self.validation_data[0])[1:])
                if self.idx < len(self.calcEpoch)-1:
                    self.idx += 1

    
    epoch_max = 200
    num_of_samples = 30
    important_epoch = np.unique(np.logspace(np.log2(1), np.log2(epoch_max), num_of_samples, dtype=int, base=2)) - 1
    calculate_recall_train = CalculateRecall(traces_train_reshaped, labels_train, 'train')
    calculate_recall_test = CalculateRecall(traces_test_reshaped, labels_test, 'test')
    calculate_act_test = CalculateActivations(important_epoch)

    callbacks = [calculate_recall_train, calculate_recall_test, save_model, calculate_act_test]

    model = create_model_mlp(classes=classes, number_samples=traces_train.shape[1])


<------------------Attacking Byte: 1------------------>


In [9]:

history = model.fit(x=traces_train_reshaped,
                    y=labels_train_categorical,
                    batch_size=500,
                    verbose=0,
                    epochs=epoch_max,
                    # class_weight=class_weight.compute_class_weight('balanced', np.unique(labels_train),
                    #                                              labels_train),
                    validation_data=(traces_test_reshaped, labels_test_categorical),
                    callbacks=callbacks)
model.save("AES_trained_model{}.h5".format(i))

# print(callbacks[4].mut)



Instructions for updating:
Use tf.cast instead.


train recall: 0.12084552115547502


test recall: 0.11939223507118582
Getting Activations...


train recall: 0.178654587747277


test recall: 0.17734642503891845
Getting Activations...


...

train recall: 0.7023434476135695


test recall: 0.6969218868869896


train recall: 0.7239489352686749


test recall: 0.7215768436057608
Getting Activations...


InternalError: Could not allocate ndarray

In [29]:

# Load Models for specific byte
tmp = 1
model.load_weights("AES_trained_model1_test.h5")


In [ ]:
print("Calculating Information...")
mut = get_information(callbacks[3].ws, traces_test, labels_test_categorical,
                                          128, 50, model, [0,0,0,0,0,0,0],calc_parallel=False)

print("Saving Mutual Information Values...")
dump(mut, 'MutInfoKey{}.gz'.format(i), compress=3)


Calculating Information...
Start calculating the information...


Calculated The information of epoch number - 0


Saving Mutual Information Values...


['MutInfoKey1.gz']

In [27]:
print(mut)

[[{'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 12.222918757079094, 'local_ITY': 1.9662671478948965}]
 [{'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 13.287712379549445, 'local_ITY': 2.529289449783519}
  {'local_IXT': 12.407872983385625, 'local_ITY': 2.1864026101384972}]
 [{'local_IXT': 13.287712379549445, 'local_I

In [ ]:

print("Plotting Mutual Info...")
I_XT_array = np.array(extract_array(mut, 'local_IXT'))
I_TY_array = np.array(extract_array(mut, 'local_ITY'))
[font_size, axis_font, bar_font, colorbar_axis, sizes, yticks, xticks, title_strs, f, axes] = load_figures(
    2, "MutualInfo")

plot_all_epochs(I_XT_array, I_TY_array, axes, important_epoch, f, 0, 0, I_XT_array.shape[0]-2, font_size, yticks, xticks,
                colorbar_axis, "Mutual Info", axis_font, bar_font, 'mut')


Plotting Mutual Info...


In [22]:
plt.show()

In [30]:
# Graph rank of correct key for one byte
tmp = 1
keyprediction = key_rank(model, inputoutput_test, traces_test_reshaped, tmp, key[0][2*tmp:2*tmp+2])

keypred = pd.DataFrame(keyprediction)

plt.plot(keypred.index.values, keypred[0], label='key{}'.format(tmp))
plt.xlabel('# of traces')
plt.ylabel('Rank')
plt.xticks(np.arange(0, 50001, step=10000))
plt.legend()
plt.title("CNN")
plt.savefig('KeyByte{}: {}.jpg'.format(tmp, key[0][2*tmp:2*tmp+2]), dpi=500, format='jpg')
plt.show()


In [12]:
# Graph all key ranks on one graph
for i in range(16):
    model.load_weights('AES_trained_model{}.h5'.format(i))
    keyprediction = key_rank(model, inputoutput_test, traces_test_reshaped, i, key[0][2*i:2*i+2])
    keypred = pd.DataFrame(keyprediction)
    plt.plot(keypred.index.values, keypred[0], label='key{}'.format(i))
plt.xlabel('# of traces')
plt.ylabel('Rank')
plt.xticks(np.arange(0, 49999, step=5000))
plt.legend()
plt.title("CNN")
plt.show()


In [ ]:
plt.plot(keypred.index.values[:1500], keypred[0][:1500], label='key{}'.format(tmp))
plt.xlabel('# of traces')
plt.ylabel('Rank')
plt.xticks(np.arange(0, 1500, step=300))
plt.legend()
plt.title("CNN")
plt.show()
